In [18]:
import pinax
from nptdms import TdmsFile, TdmsWriter, ChannelObject, GroupObject, RootObject
import matplotlib.pyplot as plt
import pandas as pd

In [19]:
## Root folder locator
import subprocess
from pathlib import Path
def get_repo_root(start: Path | None = None) -> str:

    start = start or Path.cwd()

    # --- Option 1: Try Git ---
    try:
        root = subprocess.check_output(
            ["git", "rev-parse", "--show-toplevel"],
            text=True,
            stderr=subprocess.DEVNULL,
        ).strip()
        return str(Path(root))
    except (subprocess.CalledProcessError, FileNotFoundError):
        pass  # git failed or not installed

    # --- Option 2: Walk up until `.git` is found ---
    for parent in [start] + list(start.parents):
        if (parent / ".git").exists():
            return str(parent)

    raise RuntimeError("Could not find repository root.")

In [ ]:
## TDMS Sample generation from a random file

# Source tdms file path. Modify accordingly
path = r"Z:\All Data\Cell 6 4WD\62509002 2025 Jeep Grand Cherokee 4Xe stairs_up_down_80mph_60sec_ramp_3_bag x3 w_0_3_6pct grade_2990\62509002 Test Data.tdms"

tdms_file = TdmsFile.read(path)
reduced_file_name = "69909009 Test Data"
reduced_file_path = get_repo_root()+"\\examples\\"+reduced_file_name
channel_ranges = {
    "Data" : [200,500]
}

# Create a reduced copy with only the first 100 elements of each channel
with TdmsWriter(reduced_file_path+".tdms") as writer:
    root_object = RootObject(tdms_file.properties)
    all_objects = [root_object]
    for group in tdms_file.groups():
        group_obj = GroupObject(group.name, properties=group.properties)
        all_objects.append(group_obj)
        for channel in group.channels():
            shortened_data = channel[channel_ranges.get(group.name, [0, 100])[0]:channel_ranges.get(group.name, [0, 100])[1]]  # Slice to specified size or default to 100
            if len(shortened_data) > 0:  # Only write if there's data
                all_objects.append(ChannelObject(group.name, channel.name, shortened_data, properties=channel.properties))
    writer.write_segment(all_objects)

# Read back 
reduced_tdms = TdmsFile.read(reduced_file_path+".tdms")
# Save as csv ("Data" group only)
data_group = reduced_tdms["Data"]
data_group.as_dataframe().to_csv(reduced_file_path+".csv", index=False)

In [21]:
# Comparisons

for group in tdms_file.groups():
    print(group.name, group.as_dataframe().shape, "Properties size:", len(group.properties))
print("_________________________________")
for group in reduced_tdms.groups():
    print(group.name, group.as_dataframe().shape, "Properties size:", len(group.properties))

    

RawFacilities (31023, 22) Properties size: 6
Data (30301, 535) Properties size: 21
RawVehicle (31045, 11) Properties size: 4
RawHost (9, 5) Properties size: 48
TestInfo (9, 5) Properties size: 73
RawCAN (31043, 72) Properties size: 1
RawPierburgModal (60795, 12) Properties size: 13
PhaseData (0, 3) Properties size: 0
Vspy (90045, 408) Properties size: 1
_________________________________
RawFacilities (100, 22) Properties size: 6
Data (1000, 535) Properties size: 21
RawVehicle (100, 11) Properties size: 4
RawHost (9, 5) Properties size: 48
TestInfo (9, 5) Properties size: 73
RawCAN (100, 72) Properties size: 1
RawPierburgModal (100, 12) Properties size: 13
PhaseData (0, 0) Properties size: 0
Vspy (100, 408) Properties size: 1
Vspy (90045, 408) Properties size: 1
_________________________________
RawFacilities (100, 22) Properties size: 6
Data (1000, 535) Properties size: 21
RawVehicle (100, 11) Properties size: 4
RawHost (9, 5) Properties size: 48
TestInfo (9, 5) Properties size: 73
Raw